In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as sp

pd.set_option("display.max_columns", 85)
pd.set_option("display.max_rows", 85)
sns.set_theme(context="paper", font_scale=1.5, style="ticks", rc={"axes.grid": True})

In [2]:
# Read Features Data (NEW from PP)
df = pd.read_csv("./data/features_and_response.noscale.csv", index_col=0)

# Drop NaNs
df.dropna(inplace=True)


# Collect Features and Labels
features_df = pd.DataFrame()
conf = df.drop(labels=["response", "occ_total_sum", "oldest_phylostratum"], axis=1)
features_df["occ_total_sum"] = df["occ_total_sum"]
features_df["oldest_phylostratum"] = df["oldest_phylostratum"]
features_df = pd.concat([features_df, conf], axis=1)


# Old and New response both in this DF
response = pd.read_csv("./data/old_and_new_response.csv", index_col=0)

# Read phenotype data
pheno_old_df = pd.read_csv("./data/2019_pheno.csv", index_col=0)
pheno_df = pd.read_csv("./data/2022_pheno.csv", index_col=0)

In [4]:
X = features_df.to_numpy()

### Custom Scoring: Area Under Precision Recall Curve

In [16]:
from sklearn.metrics import auc, make_scorer, precision_recall_curve

def auprc(y_true, y_scores, **kwargs):
    """ Remember to use make_scorer(auprc, needs_proba=True,) when feeding to SKL's GSCV."""
    precisions, recalls, thresholds = precision_recall_curve(y_true, y_scores)
    # results is area under x=Recall and y=Precision curve. 
    return auc(recalls, precisions)

### ML Modules

In [5]:
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from scipy.stats import sem
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

## Generic Phenotype ("phenotypic abnormality") (Old)

In [6]:
# Model as Pipeline
rf_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("rf", BalancedRandomForestClassifier(n_jobs=-1, 
                                          min_samples_leaf=10, 
                                          min_samples_split=10, 
                                          n_estimators=1000,
                                         ),
    ),
])

# Parameters
np.random.seed(2)  # Random seed

y = response["response"].to_numpy()
    
# Fit and predict 
rf_clf.fit(X, y)
pred = rf_clf.predict_proba(X)[:, 1]

In [10]:
# Save to file
gen_pheno_pred_df = pd.DataFrame(pred, index=pheno_df.index)
gen_pheno_pred_df.to_csv("./results/abnormal_phenotype_probas_2019.csv")

In [11]:
# Score (in-sample)
roc_auc_score(y, pred)

0.9376903064798611

## Generic Phenotype ("phenotypic abnormality") (New)

In [12]:
# Model as Pipeline
rf_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("rf", BalancedRandomForestClassifier(n_jobs=-1, 
                                          min_samples_leaf=10, 
                                          min_samples_split=10, 
                                          n_estimators=1000,
                                         ),),
])

# Parameters
np.random.seed(2)  # Random seed

y = response["updated_response"].to_numpy()
    
# Fit and predict 
rf_clf.fit(X, y)
pred = rf_clf.predict_proba(X)[:, 1]

In [13]:
# Save to file
gen_pheno_pred_df = pd.DataFrame(pred, index=pheno_df.index)
gen_pheno_pred_df.to_csv("./results/abnormal_phenotype_probas_2022.csv.csv")

In [14]:
# Score (in-sample)
roc_auc_score(y, pred)

0.9405071970968933

## All Phenotypes (Old)

In [17]:
# Model as Pipeline
rf_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("rf", BalancedRandomForestClassifier(n_jobs=-1, 
                                          min_samples_leaf=10, 
                                          min_samples_split=10, 
                                          n_estimators=1000,
                                         ),),
])

print("Fitting random forest to all phenotypes.\n")

# Parameters
np.random.seed(0)  # Random seed

preds = []
roc_scores = []
pr_scores = []
# Loop over phenotypes
for i in range(pheno_old_df.shape[1]):

    # Select phenotype
    y = pheno_old_df.iloc[:, i].to_numpy()
    
    # Fit and predict 
    rf_clf.fit(X, y)
    pred = rf_clf.predict_proba(X)[:, 1]
    preds.append(pred)
    
    # Score
    roc_scores.append(roc_auc_score(y, pred))
    pr_scores.append(auprc(y, pred))

    if (i%100==0): print(i)

Fitting random forest to all phenotypes.

0
100
200
300
400
500
600
700
800
900
1000
1100


In [24]:
pheno_old_pred_df = pd.DataFrame(preds, index=pheno_old_df.columns, columns=pheno_old_df.index)
pheno_old_score_df = pd.DataFrame(zip(roc_scores, pr_scores), index=pheno_old_df.columns, columns=["roc_auc", "pr_auc"])
pheno_old_pred_df.to_csv("./results/all_phenotypes_probas_2019.csv")
pheno_old_score_df.to_csv("./results/all_phenotypes_scores_2019.csv")

## All Phenotypes (New)

In [25]:
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from scipy.stats import sem
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Model as Pipeline
rf_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("rf", BalancedRandomForestClassifier(n_jobs=-1, 
                                          min_samples_leaf=10, 
                                          min_samples_split=10, 
                                          n_estimators=1000,
                                         ),),
])

print("Fitting random forest to all phenotypes.\n")

# Parameters
np.random.seed(0)  # Random seed

preds = []
roc_scores = []
pr_scores = []
# Loop over phenotypes
for i in range(pheno_df.shape[1]):

    # Select phenotype
    y = pheno_df.iloc[:, i].to_numpy()
    
    # Fit and predict 
    rf_clf.fit(X, y)
    pred = rf_clf.predict_proba(X)[:, 1]
    preds.append(pred)
    
        # Score
    roc_scores.append(roc_auc_score(y, pred))
    pr_scores.append(auprc(y, pred))
    
    if (i%100==0): print(i)

Fitting random forest to all phenotypes.

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300


In [ ]:
pheno_new_pred_df = pd.DataFrame(preds, index=pheno_df.columns, columns=pheno_df.index)
pheno_new_score_df = pd.DataFrame(zip(roc_scores, pr_scores), index=pheno_df.columns, columns=["roc_auc", "pr_auc"])
pheno_new_pred_df.to_csv("./results/all_phenotypes_probas_2022.csv")
pheno_new_score_df.to_csv("./results/all_phenotypes_scores_2022.csv")

In [4]:
scores_df = pd.read_csv("./results/all_phenotypes_scores_2022.csv", index_col=0)

scores_df.mean()

roc_auc    0.914538
pr_auc     0.292994
dtype: float64